# Crystal Quest Environment

- modeled off of https://github.com/openai/gym/blob/master/gym/envs/box2d/lunar_lander.py and
https://github.com/openai/gym/blob/master/gym/envs/atari/atari_env.py
- will put in external file when done

In [18]:
import gym

In [5]:
import sys
sys.path.append('../crystal_quest/')
import crystal_quest_env_v2 as cq
import imp 
imp.reload(cq) # reload after making changes

<module 'crystal_quest_env_v2' from '../crystal_quest/crystal_quest_env_v2.py'>

In [6]:
env = cq.Wave1Env()

In [15]:
env.observation_space
env.action_space

Discrete(5)

In [16]:
type(env.observation_space)

gym.spaces.box.Box

In [19]:
type(env.observation_space) == gym.spaces.Box

True

In [7]:
obs = env.reset()
env.ship_location

array([20, 20])

In [21]:
type(env.action_space)      == gym.spaces.Discrete

True

In [22]:
env

# Crystal Quest Rendering Environment (Simplest)

In [13]:
# randomly sampling actions and visualizing 
env = cq.Wave1Env()
obs = env.reset()
for i in range(1000):
    obs,r,done = env.step(env.action_space.sample())
    env.render()
env.render(close=True)

In [23]:
    benchmark = gym.benchmark_spec('Atari40M')

    # Change the index to select a different game.
    task = benchmark.tasks[3]

In [30]:
#obs[:,:,1]

In [25]:
task.max_timesteps

40000000